In [1]:
import pandas as pd
nearmap_df_1 = pd.read_parquet("/data/finetune_mtmv_nearmap_rg/chipping/chips/dataset.parquet")
nearmap_df_1["chips_path"] = "/data/finetune_mtmv_nearmap_rg/chipping/chips"
nearmap_df_2 = pd.read_parquet("/data/evaluate_mtmv_nearmap_rg/chips/dataset.parquet")
nearmap_df_2["chips_path"] = "/data/evaluate_mtmv_nearmap_rg/chips"
fra_df = pd.read_parquet("/data/evaluate_mtmv_fra_rg/chips/dataset.parquet")
fra_df["chips_path"] = "/data/evaluate_mtmv_fra_rg/chips"
nzl_df = pd.read_parquet("/data/evaluate_mtmv_nzl_rg/chips/dataset.parquet")
nzl_df["chips_path"] = "/data/evaluate_mtmv_nzl_rg/chips"


In [2]:
df = pd.concat([nearmap_df_1, nearmap_df_2], axis=0)
df = df.drop_duplicates(subset=["geometry",'imagery_date', 'imagery_source'])

In [3]:
val_fraction = 0.4

# Split train, val, test (after grouping by datarow_id!)
from sklearn.model_selection import GroupShuffleSplit 

splitter = GroupShuffleSplit(test_size=val_fraction, n_splits=1, random_state = 7)
split = splitter.split(df, groups=df['geometry'])
train_inds, val_inds = next(split)


X_train_nearmap = df.iloc[train_inds]
X_val_nearmap = df.iloc[val_inds]


In [4]:
rg_labels = list(X_train_nearmap["geometry_labels"].unique())

In [5]:
import os
base_dir = "/cnvrg/rg_evaluation/train"
for label in rg_labels:
    os.makedirs(os.path.join(base_dir, label), exist_ok=True)

base_dir = "/cnvrg/rg_evaluation/val"
for label in rg_labels:
    os.makedirs(os.path.join(base_dir, label), exist_ok=True)


In [6]:
len(X_val_nearmap)

22212

In [7]:
X = X_val_nearmap

dataset_path = "/cnvrg/rg_evaluation/val"

for single_file, pred, chips_dir in zip(X.filename, X.geometry_labels, X.chips_path):
    image_full_path = os.path.join(chips_dir, single_file)
    filename = image_full_path.split("/")[-1]   
    os.symlink(image_full_path, os.path.join(dataset_path, pred, filename))


In [8]:
len(X_train_nearmap)

33318

In [9]:
X = X_train_nearmap

dataset_path = "/cnvrg/rg_evaluation/train"

for single_file, pred, chips_dir in zip(X.filename, X.geometry_labels, X.chips_path):
    image_full_path = os.path.join(chips_dir, single_file)
    filename = image_full_path.split("/")[-1]   
    os.symlink(image_full_path, os.path.join(dataset_path, pred, filename))


In [10]:
X_val_nearmap.columns

Index(['attribute_geometry_id', 'contact_id', 'geometry', 'geometry_id',
       'geometry_labels', 'imagery_date', 'imagery_source', 'label_vote_id',
       'label_vote_status', 'datarow_id', 'filename', 'entry_hash',
       'chips_path', 'origin_mnemonic'],
      dtype='object')